# Homework 5

By Yanhao Miao

## Question 1 Algorithmic questions

Find the kth largest element in an unsorted array. Note that it is the kth largest element in the sorted order, not the kth distinct element.

Here, I use a minimum heap of size k. As we iterate the array, when the heap hasn't been full, we directly add an element to the heap (we need to sift up). After the heap is full, we compare the new element with the top of this heap (the current minimum). If the new element is greater than the top element, we pop out the top and add the new element, then sift down to maintain the heap order. Otherwise, we just move on to the next element. In this way, we can achieve the kth largest element by simply retrieve the top element of the heap after the tranversal. The time complexity will be O(nlog(k)), because the 'sift' algorithm requires O(log(k)) and suppose there are totally n elements in the array. 

In [30]:
class Heap(object):
    # create a class to implement a minimum heap, use array structure
    
    def __init__(self, max_size):
        # here we set a guard in the position 0, to simplify the sift algo
        # then the heap array starts from index 1
        self.heap = [float("-inf")]  
        self.max_size = max_size

    def __len__(self):
        # the length should not include index 0
        return len(self.heap) - 1

    def siftup(self):
        # to make the last element 'sift up' to its correct position
        
        pos = len(self)                     
        x = self.heap[-1] # retrieve the value of the last element  
        
        # when the index is 0, x is surely smaller than math.inf, exit the loop
        while x < self.heap[int(pos/2)]:
            self.heap[pos] = self.heap[int(pos/2)]
            pos = int(pos/2)
        self.heap[pos] = x

    def siftdown(self, idx):
        # to make the element with index 'idx' to its correct position
        
        temp = self.heap[idx]
        length = len(self)
        
        while True:
            child_idx = idx * 2 
            if child_idx > length:
                break

            # compare the current element with its right child (if exists), otherwise its left child
            if child_idx != length and self.heap[child_idx] > self.heap[child_idx + 1]:
                child_idx += 1
            if temp > self.heap[child_idx]:
                self.heap[idx] = self.heap[child_idx]
            else:
                break  
            idx = child_idx
        # swap
        self.heap[idx] = temp
        
    def top(self):
        # obtain the top element
        if not len(self):
            print("Empty Heap!")
        return self.heap[1]

    def update(self, idx, key):
        # update the position 'idx' to be 'key'
        if idx > len(self) or idx < 1:
            print("Index out of range!")
            return
        self.heap[idx] = key
        self.siftdown(idx)
    
    def insert(self, val):
        # insert a value to the heap
        
        # check whether there are enough space
        if not len(self) + 1 <= self.max_size:
            print("Heap overflow!")
            return
        
        (self.heap).append(val)
        self.siftup()


In [37]:
# define the function
def get_largest_num(nums,k):
    myheap = Heap(k)
    
    for x in nums[0:k]:
        myheap.insert(x)
    
    for x in nums[k:]:
        if myheap.top() < x:
            myheap.update(1, x)
    
    return myheap.top()

To use heapq:

In [36]:
import heapq

def get_largest_num2(nums, k):
    
    if k > len(nums) or k < 1:
        print("Index out of range!")
    
    heap = nums[:k]
    heapq.heapify(heap)
    
    for num in nums[k:]:
        if num > heap[0]:
            heapq.heappushpop(heap, num)
    
    # Heap root is the kth largest element
    return heap[0]

In [46]:
# Test cases for the first version

# the sample case provided in the problem
print(get_largest_num([3,2,3,1,2,4,5,5,6],4))
print(get_largest_num([3,2,1,5,6,4], 2))

# only one element
print(get_largest_num([7], 1))

# all the elements are the same
nums = [1 for i in range(10000)]
print(get_largest_num2(nums, 100))

# large number of elements, and compare with heapq.nlargest
import random

nums = [random.randint(1, 10000) for i in range(10000)]
k = 500
print(get_largest_num(nums, k))
print(heapq.nlargest(k, nums)[-1])

4
5
7
1
9490
9490


In [47]:
# Test cases for the second version

# the sample case provided in the problem
print(get_largest_num2([3,2,3,1,2,4,5,5,6],4))
print(get_largest_num2([3,2,1,5,6,4], 2))

# only one element
print(get_largest_num2([7], 1))

# all the elements are the same
nums = [1 for i in range(10000)]
print(get_largest_num2(nums, 100))

# large number of elements, and compare with heapq.nlargest
import random

nums = [random.randint(1, 10000) for i in range(10000)]
k = 500
print(get_largest_num2(nums, k))
print(heapq.nlargest(k, nums)[-1])

4
5
7
1
9503
9503


# Question 2 Multithreading pop

Using python data structure Queue, generate a queue that have 10000 elements. Then use 1, 5, 20 threads to pop out the elements out. Compare the efficiency when you use different number of threads and explain why you see the pattern of performance.

In [49]:
import threading
import queue
import time

# generate a queue that have 10000 elements
def generate_q():
    q = queue.Queue()
    for num in range(10000):
        q.put(num)
    return q

def thread_function(name,thread_num):
    tasks = q.qsize()
    for i in range(tasks // thread_num):
        q.get()

In [51]:
def multithread_pop(q, thread_num):
    # create a list to store all the threads
    threads = []
    for i in range(thread_num):
            # set up a thread
            t = threading.Thread(target = thread_function, args=(q, thread_num))
            threads.append(t)
            t.start()
    # collect the result    
    for t in threads:
            t.join()
            
q = generate_q()
%time multithread_pop(q, 1)
q = generate_q()
%time multithread_pop(q, 5)
q = generate_q()
%time multithread_pop(q, 20)

CPU times: user 20.5 ms, sys: 886 µs, total: 21.4 ms
Wall time: 20.8 ms
CPU times: user 12.5 ms, sys: 420 µs, total: 12.9 ms
Wall time: 12.8 ms
CPU times: user 13.8 ms, sys: 941 µs, total: 14.7 ms
Wall time: 14.5 ms


It seems that python is not very efficient at multi-thread programming. Because if we continuously increase the number of threads, the efficiency doesn't improve much or even lower.